2021-02-12

# **XGBScore: A Gradient Boosted Decision Tree Scoring Function for Structure Based Virtual Screening**

This is my lab book for my dissertation project. It will contain my daily work towards the project, and *in silico* experiments performed in python 3 code cells for debugging and evaluation before being turned into separate scripts for publication and analysis.

### **Table of Contents**
- Database Scraping
- Dataset Assembly
- Dataset Cleaning
- Feature Engineering
- Adding Decoys
- Splitting the Data
- Feature Importance/Dimensionality Reduction
- Training the Model
- Optimising the Model
- Evaluating the Model

# **Database Scraping**

While there will likely be lots of redundancy, four databases are being scraped for data for the training and test sets. These are:
- Binding MOAD
- PDBBind
- Binding DB
- DUD-E

Generally, the approach will be to download the raw data into a folder called 'Dissertation/Data/Raw/{database_name}/Compressed', e.g. for Binding_MOAD: 'Dissertation/Data/Raw/Binding_MOAD/Compressed'. Then, the files will be extracted into 'Dissertation/Data/Raw/Binding_MOAD/Extracted'.

## **Binding MOAD**

Binding MOAD offers several different datasets. I have downloaded the structures (biounits) and binding data for the [**Non-redundant dataset only with binding data**](https://bindingmoad.org/Home/download). This dataset contains all complexes which have known binding data, and includes one 'leader' from each family with similar binding to prevent very similar data clogging the dataset. 

It has been downloaded to 'Dissertation/Data/Raw/Binding_MOAD/Compressed' and extracted to 'Dissertation/Data/Raw/Binding_MOAD/Extracted'. I will look at downloading the larger set once the pipeline for raw data to clean training and test datasets has been put together. 


The Binding MOAD files are stored as 'Biounit' files or .bio files, which are just partial structure PDB files and can be treated as shuch by changing the extension from '.bio1' to '.pdb'. We should download the original PDB files to a third folder 'Dissertation/Data/Raw/Binding_MOAD/original_PDB_files' just in case they are more useful. The code below downloads them:

In [ ]:
import os
import requests
from tqdm import tqdm

def create_target_url(filepath):
    
    # get the pdb code from the filename
    target_name = filepath.split('.')[0]
    
    # set the url string using the pdb code as where to download the pdb file
    target_url = f'https://files.rcsb.org/download/{target_name}.pdb'
    return target_url, target_name

def save_target_file(url):
    
    # get the file url and the target name
    url, target_name = create_target_url(url)
    
    # change this as to where you need to save the pdb files
    target_path = f'/home/milesm/Dissertation/Data/Raw/Binding_MOAD/original_PDB_files/{target_name}.pdb'
    response = requests.get(url)
    
    # ping the pdb and download the file if the url exists
    if response.status_code == 200:
        with open(target_path, 'wb') as file:
            file.write(response.content)
            file.close()
    else:
        print(f'Whoops! Somethings wrong: Response {response.status_code}')


# get the list of all the Binding_MOAD extracted protein-ligand complex biounit files
filepaths = os.listdir('/home/milesm/Dissertation/Data/Raw/Binding_MOAD/Extracted/BindingMOAD_2020')

# download the pdb files for all the Binding_MOAD extracted protein-ligand complex biounit files
with tqdm(total=len(filepaths)) as pbar:
    for target_file in filepaths:
        save_target_file(target_file)
        pbar.update(1)

## PDBBind

I have downloaded the ['Protein-ligand complexes: The refined set'](http://www.pdbbind.org.cn/download.php) to the standard directory 'Dissertation/Data/Raw/PDBBind/Compressed', and extracted it to 'Dissertation/Data/Raw/PDBBind/Extracted'. No more needed to be done to the PDBBind dataset.

## Binding DB

Cannot see a way of differentiating only those complexes which have crystal structures attached to them. At this stage is is not being included in the project.

## DUD-E

For some reason, when I try to download the whole dataset at once, the server is throwing a 503/overload saying I've exceeded the maximum number of simultaneous downloads, despite it only being one tarball file. I think we can scrape the files one by one fairly quickly as there are only 102 of them. The files downloaded will be [these ones, the compressed file of the receptor and all actives and decoys in the individual target directory, like aa2ar.tar.gz](http://dude.docking.org/targets/aa2ar). Each file has a standard base url of 'dude.docking.org/targets/{target}/{target.tar.gz}'. So if we scrape all the hrefs from the main index page, we can just populate a list and ping them one at a time:

In [ ]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

# set the index url
index_url = 'http://dude.docking.org/subsets/all'

def create_target_url(url):
    
    # this creates the url string where the file should logically be stored on DUD-E
    position = url.find('/targets/') + len('/targets/')
    target_name = url[position:]
    target_url = f'http://dude.docking.org/targets/{target_name}/{target_name}.tar.gz'
    
    # returns the url and the target protein name for use as the master folder name
    return target_url, target_name

def save_target_file(url):
    
    # get the url and the name for the folder to store the complexes in
    url, target_name = create_target_url(url)
    
    # change this path to where you want the files to be downloaded to
    target_path = f'/home/milesm/Dissertation/Data/Raw/DUD-E/Compressed/{target_name}.tar.gz'
    
    # ping the url and check it exists
    response = requests.get(url, verify=False)
    if response.status_code == 200:
        
        # save the compressed archive file with the decoys, actives and receptor in
        with open(target_path, 'wb') as file:
            file.write(response.content)
            file.close()
    else:
        print(f'Whoops! Somethings wrong: Response {response.status_code}')

# check the DUD-E homepage for the HTML and make it readable with bs4
index_page = requests.get(index_url)
html_content = index_page.text
soup = BeautifulSoup(html_content, 'html.parser')

# get urls of all the target proteins avaliable
file_urls = [url['href'] for url in soup.find_all('a') if '/targets/' in str(url)]

# for each target protein, download all the associated files
with tqdm(total=len(file_urls)) as pbar:
    for target_file in file_urls:
        save_target_file(target_file)
        pbar.update(1)

2021-02-13

DUD-E files have been downloaded in a separate tar.gz file for each target protein. These all need extracting, which can be performed by the script below:

In [ ]:
import tarfile
import os
from tqdm import tqdm

# simple function to extract a file
def extract_file(fname):
    tar = tarfile.open(fname, "r:gz")
    tar.extractall()
    tar.close()

# make a list of all the filepaths of the compressed protein target files in the folder
files = [('/home/milesm/Dissertation/Data/Raw/DUD-E/Compressed/' + file) for file in os.listdir('/home/milesm/Dissertation/Data/Raw/DUD-E/Compressed/')]

# extract all the files
with tqdm(total=len(files)) as pbar:
    for file in files:
        extract_file(file)
        pbar.update(1)

These can then be sorted by filetype, and the extracted folders copied and pasted into the 'Dissertation/Data/Raw/DUD-E/Extracted' folder.

# **Data Cleaning**

2021-02-18

## Software Installations

All the software outlined below is needed for the steps of producing a csv file of features from each protein-ligand complex and decoys from DUD-E

#### **PyMOL 2.4**

Downloaded the tar.bz2 file from [PyMOL](https://pymol.org/2/), unpacked it and ran pymol using: 

#### **BINANA**

This is just a python2 script, and can be downloaded from [Professor Jacob Durrants Gitlab](https://durrantlab.pitt.edu/binana/), extracted and executed when needed.

#### **Autodock and Autodocktools via MGLTools 1.5.6**

Downloaded the x64 GUI linux installer from the official source and executed it.

#### **Openbabel 3.1.1**

I have had to build openbabel several times from source with different flags for cmake before getting it working - what worked was installing:
- libopenbabel-dev
- libopenbabel4v5
- openbabel-gui

Using the command:

And then cloning openbabel from github and building from source using the instructions from the documentation with specific flags for the python bindings when using cmake:

#### **Python Libraries**

All python libraries except openbabel were pip installed with "python3 -m pip install xxx"
- BioPython
- Open Drug Discovery Toolkit (ODDT)
- Pandas
- Numpy
- Biopandas

2021-02-20

## **General Approach**

The general approach for each cleaning each dataset and the end goals of the data cleaning can be seen below in Figure 1. The general ideal will be to clean data created in the last step from the 'Dissertation/Data/Raw/{database_name}/Extracted' directory into a standard format of:
- A foldername with the pdb code containing:
    - A ligand.pdb file
    - A receptor.pdb file

These folders will be stored in a new folder: 'Dissertation/Data/Parsed/{database_name}'

<img src='Images/data_cleaning.png' align='center'/>

*Figure 1 - Flowchart of planned data cleaning process*

2021-02-21

## **DUD-E Data Cleaning**

### **Splitting active and decoy ligand multimol files**

The DUD-E ligands and actives are stored in multimol .mol2 files, which need to be split into separate .mol2 files for GWOVina docking. This has been done with a custom python script which has been pushed to the github as 'split_ligands.py'. See the code comments for how it works. This script produces parsed DUD-E data in the form:

- Foldername is protein target:
    - 'actives' folder contains one mol2 file for each active ligand
    - 'decoys' folder contains one mol2 file for each decoy
    - crystal_ligand.mol2 file is a docked ligand
    - receptor,pdb is the receptor pdb file

### **Docking active and decoy ligand mol2 files to target protein with GWOVina**

The supplied actives and decoys do not appear to be docked for the DUD-E database. Therefore, each one needs docking with GWOVina before being converted to a PDBQT file with Autodocktools. This script is a work in progress, but it will automate the docking process and create a new subset of DUD-E data in 'Dissertation/Data/Parsed/DUD-E/Docked' with a folder for each protein-ligand and protein-decoy complex.

2021-02-22

## **PDBBind Data Cleaning**

The PDBBind Database ships in the ideal format for this project, and so no data cleaning needs to be undertaken for this dataset. It has been copied directly from the 'Dissertation/Data/Raw/PDBBind/Extracted' to the 'Dissertation/Data/Parsed/PDBBind' folder.

## **Binding MOAD Data Cleaning**

### **Splitting Protein-Ligand Complexes**

The raw extracted Binding MOAD dataset just contains .pdb complex files, which need to be separated into a 'protein.pdb' file and a 'ligand.pdb' file. In .pdb files, amino acids/residues are stored as type 'ATOM', whereas non-amino acid residues are stored as type 'HETATM'. This 'HETATM' type includes sulfates, zincs and waters that are likely not ligands, as well as cofactors. 

Therefore, I have written a python script called 'split_complex.py' and pushed it to the project github. This looks for all the different residues where all the atoms are classed as 'HETATM', and then counts how many atoms are present in the residue, and keeps the longest 'HETATM' residue or ligand. This way, the chemical ligand is kept, as waters, zincs and sulfates have very few atoms compared to standard ligands. **This approach misses peptide ligands and this will need to be addressed ASAP. It also makes duplicate folders in the instance of some ligands due to multiple biounit files being present and one throwing an error**. The script produces a 'protein.pdb' and a 'ligand.pdb' file in a folder named as the complex pdb code in the master folder 'Dissertation/Data/Parsed/Binding_MOAD/'

To resolve the above errors, I have written a Binding MOAD specific splitter named 'split_MOAD_complex.py'. This loads the binding data csv from Binding MOAD, and uses this to find the ID for the active ligand in the complex, then splits it and saves it separately from the protein, as well as stripping all the other HETATM types such as sulfates and waters. This script produces a folder for each biounit file called '/home/milesm/Dissertation/Data/Parsed/Binding_MOAD/{pdb_code}-X', where X is the biounit extension (e.g. .bio1 would be 1). Folders with only the protein file in are produced when the ligand was a peptide, or when no ligand was present in the particular biounit file.

2021-03-01

# **Removing Duplicate Data Entries**

Since PDBBind ships in the ideal format, and DUD-E actives are not docked, the order of preference for keeping structures in the case of duplicates is as follows:

1. PDBBind Structures
2. Binding MOAD Structures
3. DUD-E Active Structures

There is a difficult step involved in this part, as **DUD-E protein-ligand complexes are not labelled by PDB code but by CHEMBLID and Protein Target name**. Therefore, I will need to think of a way to remove duplicates after these have been docked.

For PDBBind vs Binding MOAD, the duplicates can be stripped and the full non-redundant set copied  using some very simple python3 code:

In [ ]:
import os
import shutil
from tqdm import tqdm

pdbbind_structures = os.listdir('/home/milesm/Dissertation/Data/Parsed/PDBBind')

binding_MOAD_structures = os.listdir('/home/milesm/Dissertation/Data/Parsed/Binding_MOAD')

pdbbind_clean_structures = [structure.upper() for structure in pdbbind_structures]
binding_MOAD_clean_structures = list(set(list([structure.split('_')[0].upper() for structure in binding_MOAD_structures])))

MOAD_exclusives = list()

for structure in binding_MOAD_clean_structures:
    if structure not in pdbbind_clean_structures:
        MOAD_exclusives.append(structure)

print(f'{len(MOAD_exclusives)} structures exclusive to MOAD')

os.mkdir('/home/milesm/Dissertation/Data/Parsed/Non_redundant')
os.mkdir('/home/milesm/Dissertation/Data/Parsed/Non_redundant/PDBBind')
os.mkdir('/home/milesm/Dissertation/Data/Parsed/Non_redundant/Binding_MOAD')

pdbbind_structures_filepaths = [f'/home/milesm/Dissertation/Data/Parsed/PDBBind/{file}' for file in pdbbind_structures]

MOAD_exclusives_filenames = list()
for pdb_code in MOAD_exclusives:
    for filename in binding_MOAD_structures:
        pdb_code_upper = pdb_code.upper()
        filename_upper = filename.upper()
        if pdb_code_upper in filename_upper:
            MOAD_exclusives_filenames.append(filename)
            
print(f'Found {len(MOAD_exclusives_filenames)} exclusive MOAD files for copying')

binding_MOAD_exclusive_structures_filepaths = [f'/home/milesm/Dissertation/Data/Parsed/Binding_MOAD/{file}' for file in MOAD_exclusives_filenames]

with tqdm(total=len(pdbbind_structures)) as pbar:
    for filepath, file in zip(pdbbind_structures_filepaths, pdbbind_structures):
        shutil.copytree(filepath, f'/home/milesm/Dissertation/Data/Parsed/Non_redundant/PDBBind/{file}')
        pbar.update(1)


with tqdm(total=len(MOAD_exclusives_filenames)) as pbar:
    for filepath, file in zip(binding_MOAD_exclusive_structures_filepaths, MOAD_exclusives_filenames):
        shutil.copytree(filepath, f'/home/milesm/Dissertation/Data/Parsed/Non_redundant/Binding_MOAD/{file}')
        pbar.update(1)

## **Conversion from .pdb to .pdbqt using Autodocktools**

All the datasets should now be in the same ligand.pdb and protein.pdb format, with duplicates removed, in the 'Dissertation/Data/Parsed/Master' folder. The next step is to convert these pdb files into pdbqt files using autodocktools. This is a command line process. Autodocktools ships with two python scripts:



These scripts convert pdb files to pdbqt files. Here, I will use the following commands to convert the pdb files to pdbqt files:

For the ligand pdb file, this command adds hydrogens and gasteiger charges:

For the protein pdb file, this command adds hydrogens and gasteiger charges, and removes any remaining waters:

I have written a python 3 script 'pdbqt_batch_converter.py' which takes all the protein and ligand pdb files in the 'Dissertation/Data/Parsed/Non-redundant' folders, converts them using the autodocktools command line commands above and then saves them in the 'Dissertation/Data/PDBQT/Non-redundant' folder. It has been pushed to the project github. **Currently, this script does not work for some Binding MOAD files, as they either have peptide ligands, or have two ligands in one binding pocket which are stored as one single ligand in the PDB file. This confuses autodock when it cannot find a bond between them, and it ditches/doesn't save half the ligand.**

2021-03-02

## **Adding Features for Each Complex**

Once a master dataset of clean pdbqt files has been produced by 'pdbqt_batch_converter.py', each protein.pdbqt and ligand.pdbqt needs to be fed through 'binana.py' which calculates the following features:

- Atom-type pair counts within 2.5 angstroms
- Atom-type pair counts within 4.0 angstroms
- Ligand atom types
- Summed electrostatic energy by atom-type pair, in J/mol
- Number of rotatable bonds in the ligand
- Active-site flexibility
- Hydrogen bonds
- Hydrophobic contacts (C-C)
- pi-pi stacking interactions
- T-stacking (face-to-edge) interactions
- Cation-pi interactions
- Salt Bridges

This script is used in the following way in the command line to add the above features to a protein-ligand complex:

This command outputs a text file of information about the protein-ligand complex comtaining markdown tables. These output text files will be stored in a separate directory, 'Dissertation/Data/BINANA/'. Since the information is not readily convertable to .csv format, I am writing the python 3 script 'parse_binana_output.py' to read in the output text file, find the information and output a csv file with a single row. 